In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation,Embedding,LSTM, TimeDistributed,Bidirectional
from keras.optimizers import RMSprop,SGD
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


Lecture des données
===================

In [2]:
def read_conll_sentence(istream):
    x_seq = []
    y_seq = []
    line = istream.readline()
    while line and not line.isspace():
        fields = line.split()
        x_seq.append(fields[1])
        y_seq.append(fields[3])
        line = istream.readline()
    return (x_seq,y_seq)

In [3]:
def read_conll_corpus(filename):
    X = []
    Y = []
    istream = open(filename)
    (x,y) = read_conll_sentence(istream)
    while x and y:
        X.append(x)
        Y.append(y)
        (x,y) = read_conll_sentence(istream)
    istream.close()
    return X,Y

In [4]:
X,Y = read_conll_corpus('../projets-2017-2018/sequoia-corpus.np_conll')

In [5]:
print(X[:2],Y[:2])

[['Gutenberg'], ['Cette', 'exposition', 'nous', 'apprend', 'que', 'dès', 'le', 'XIIe', 'siècle', ',', 'à', 'Dammarie-sur-Saulx', ',', 'entre', 'autres', 'sites', ',', 'une', 'industrie', 'métallurgique', 'existait', '.']] [['N'], ['D', 'N', 'CL', 'V', 'C', 'P', 'D', 'A', 'N', 'PONCT', 'P', 'N', 'PONCT', 'P', 'A', 'N', 'PONCT', 'D', 'N', 'A', 'V', 'PONCT']]


Codage des données
==================

In [6]:
x_set = set([])
y_set = set([])
init_token = "__START__"
for x in X:
    x_set.update(x)
for y in Y:
    y_set.update(y)
rev_x_codes = [init_token]
rev_x_codes.extend(list(x_set))
rev_y_codes = list(y_set)
x_codes     = dict((x,idx) for idx,x in enumerate(rev_x_codes))
y_codes     = dict((y,idx) for idx,y in enumerate(rev_y_codes))
print(y_codes)



{'PRO': 0, 'P+D': 1, 'D': 6, 'ADV': 7, 'CL': 3, 'N': 4, 'C': 5, 'A': 8, 'V': 2, 'P': 10, 'PONCT': 9, 'ET': 13, 'PREF': 11, 'P+PRO': 12, 'I': 14}


In [7]:
Xcodes = []
for x in X:
    Xcodes.append([x_codes[elt] for elt in x])
Ycodes = []
for y in Y:
    ymat = np.zeros((len(y),len(y_codes)))
    for idx,elt in enumerate(y):
        ymat[idx,y_codes[elt]] = 1.0
    Ycodes.append(ymat)

In [8]:
print(Xcodes[1],Ycodes[1])

[1029, 5424, 3127, 6400, 7968, 4476, 2242, 2327, 3928, 5534, 746, 1690, 5534, 8991, 9792, 7551, 5534, 3441, 9673, 4696, 4506, 8872] [[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.

Padding et troncation...
-------------------------

In [9]:
L = [len(y) for y in Ycodes]
mL = int(sum(L)/len(L))
print(mL) #longueur moyenne
Xcodes = pad_sequences(Xcodes,maxlen=mL)
Ycodes = pad_sequences(Ycodes,maxlen=mL)

21


Structure du modèle
===================

In [10]:
x_size = len(x_codes)
y_size = len(y_codes)
embedding_size = 50
memory_size    = 50
model = Sequential()
model.add(Embedding(x_size,embedding_size))
model.add(Bidirectional(LSTM(memory_size,return_sequences=True))) #bi-LSTM
#model.add(LSTM(memory_size,return_sequences=True))               #simple LSTM
model.add(TimeDistributed(Dense(y_size, activation='softmax'))) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          518950    
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 100)         40400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 15)          1515      
Total params: 560,865
Trainable params: 560,865
Non-trainable params: 0
_________________________________________________________________


Descente de gradient
====================

In [11]:
sgd = RMSprop(lr=0.001)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(Xcodes,Ycodes,epochs=20,batch_size=32)

Epoch 1/20
3099/3099 [==============================] - 6s - loss: 1.4800 - acc: 0.2347     
Epoch 2/20
3099/3099 [==============================] - 6s - loss: 0.9089 - acc: 0.4838     
Epoch 3/20
3099/3099 [==============================] - 6s - loss: 0.4586 - acc: 0.5956     
Epoch 4/20
3099/3099 [==============================] - 6s - loss: 0.2731 - acc: 0.6548     
Epoch 5/20
3099/3099 [==============================] - 6s - loss: 0.1756 - acc: 0.6819     
Epoch 6/20
3099/3099 [==============================] - 6s - loss: 0.1170 - acc: 0.6968     
Epoch 7/20
3099/3099 [==============================] - 6s - loss: 0.0808 - acc: 0.7058     
Epoch 8/20
3099/3099 [==============================] - 6s - loss: 0.0577 - acc: 0.7120     
Epoch 9/20
3099/3099 [==============================] - 6s - loss: 0.0428 - acc: 0.7155     
Epoch 10/20
3099/3099 [==============================] - 6s - loss: 0.0329 - acc: 0.7178     
Epoch 11/20
3099/3099 [==============================] - 6s - loss: 0

Prédictions 
===========

(Attention !) Predictions sur les données d'entrainement...

In [13]:
def eval_model(x_data,y_ref):
    
    C = 0
    N = 0
    for x,yvec in zip(x_data,y_ref):
        prob_vec = model.predict(x)
        L = [np.argmax(tok_probs) == np.argmax(y) for(y,tok_probs,tok_code) in zip(yvec,prob_vec,x) if tok_code != 0]
        C += sum(L)
        N += len(L)
        
    return C/N

eval_model(Xcodes,Ycodes)    

0.96167365434294672